In [1]:
import molmap
import os

In [32]:
metric = 'cosine'
method = 'tsne'
n_neighbors = 30
min_dist = 0.1
bitsinfo = molmap.feature.fingerprint.Extraction().bitsinfo
bitsinfo.Subtypes.unique()

array(['MorganFP', 'RDkitFP', 'AtomPairFP', 'TorsionFP', 'AvalonFP',
       'EstateFP', 'MACCSFP', 'PharmacoErGFP', 'PharmacoPFP', 'PubChemFP',
       'MHFP6', 'MAP4'], dtype=object)

In [39]:
flist = bitsinfo[bitsinfo.Subtypes.isin(["MAP4"])].IDs.tolist()
mp2 = molmap.MolMap(ftype = 'fingerprint', metric = metric, flist =flist,)
flist = mp2.scale_info['var'].sort_values().index.tolist()

In [40]:
mp2 = molmap.MolMap(ftype = 'fingerprint', metric = metric, flist =flist,)
mp2.fit(method = method, n_neighbors = n_neighbors, min_dist = min_dist) 

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2048 samples in 0.002s...
[t-SNE] Computed neighbors for 2048 samples in 0.108s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2048
[t-SNE] Computed conditional probabilities for sample 2000 / 2048
[t-SNE] Computed conditional probabilities for sample 2048 / 2048
[t-SNE] Mean sigma: 0.173238
[t-SNE] Computed conditional probabilities in 0.102s
[t-SNE] Iteration 50: error = 157.6343536, gradient norm = 0.1600981 (50 iterations in 1.249s)
[t-SNE] Iteration 100: error = 164.7572479, gradient norm = 0.1811365 (50 iterations in 1.034s)
[t-SNE] Iteration 150: error = 177.7385864, gradient norm = 0.1964027 (50 iterations in 1.002s)
[t-SNE] Iteration 200: error = 170.7337494, gradient norm = 0.1697190 (50 iterations in 1.254s)
[t-SNE] Iteration 250: error = 177.8373718, gradient norm = 0.1224262 (50 iterations in 0.862s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 177.837372
[t-SNE] Iteration 300

In [41]:
mp2.plot_scatter()

2020-09-01 23:33:17,377 - INFO - [bidd-molmap] - generate file: ./fingerprint_2048_cosine_tsne_scatter
2020-09-01 23:33:17,429 - INFO - [bidd-molmap] - save html file to ./fingerprint_2048_cosine_tsne_scatter


In [42]:
from molmap.model import RegressionEstimator, MultiClassEstimator, MultiLabelEstimator
from molmap import loadmap, dataset,feature
from molmap.show import imshow_wrap
from chembench import load_data

from sklearn.utils import shuffle 
from joblib import load, dump
import numpy as np
import pandas as pd
import os

bitsinfo = feature.fingerprint.Extraction().bitsinfo
fp_types = bitsinfo.Subtypes.unique()

esol = dataset.load_ESOL(True)
lipop = dataset.load_Lipop(True)
#malaria = dataset.load_malaria(True)

data = esol #malaria

task_name = data.task_name
task_type = data.task_type
_, induces = load_data(task_name)
smiles = data.x
Y = data.y

X2 = mp2.batch_transform(smiles, n_jobs = 16)

Regression_res = []
for i, idx in enumerate(induces):
    train_idx, valid_idx, test_idx = idx

    X = X2[train_idx]
    y = Y[train_idx]

    X_valid = X2[valid_idx]
    y_valid = Y[valid_idx]

    X_test = X2[test_idx]
    y_test = Y[test_idx]          

    fmap_shape1 = X.shape[1:]
    n_outputs = Y.shape[1]

    clf = RegressionEstimator(n_outputs=n_outputs,  
                              fmap_shape1 = fmap_shape1, 
                              dense_layers = [128, 64],
                              gpuid = 3,
                             ) 

    clf.fit(X, y, X_valid, y_valid)

    train_rmses, train_r2s = clf._performance.evaluate(X,y)
    valid_rmses, valid_r2s = clf._performance.evaluate(X_valid,y_valid)            
    test_rmses, test_r2s = clf._performance.evaluate(X_test,y_test)


    final_res = {'task_name':task_name, 
                 'repeat_id':i,
                 'train_rmse':np.nanmean(train_rmses), 
                 'valid_rmse':np.nanmean(valid_rmses),                      
                 'test_rmse':np.nanmean(test_rmses),

                 'train_r2':np.nanmean(train_r2s), 
                 'valid_r2':np.nanmean(valid_r2s),                      
                 'test_r2':np.nanmean(test_r2s),                        
                }

    Regression_res.append(final_res)


total samples: 1128


  0%|          | 0/1128 [00:00<?, ?it/s]

total samples: 4200
loading dataset: ESOL number of split times: 3


 85%|########5 | 964/1128 [00:41<00:05, 28.00it/s]

error when calculating C


100%|##########| 1128/1128 [00:47<00:00, 23.75it/s]


RegressionEstimator(batch_size=128, conv1_kernel_size=11, dense_avf='relu',
          dense_layers=[128, 64], epochs=800, gpuid='3', loss='mse',
          lr=0.0001, monitor='val_loss', name='Regression Estimator',
          patience=50, random_state=32, verbose=2)
epoch: 0001, loss: 12.5242 - val_loss: 11.5484; rmse: 3.2158 - rmse_val: 3.4129;  r2: 0.2545 - r2_val: 0.1891                                                                                                    
epoch: 0002, loss: 9.3012 - val_loss: 8.5768; rmse: 2.7230 - rmse_val: 2.9416;  r2: 0.2541 - r2_val: 0.1912                                                                                                    
epoch: 0003, loss: 6.1895 - val_loss: 5.1648; rmse: 2.0383 - rmse_val: 2.2822;  r2: 0.2495 - r2_val: 0.1887                                                                                                    
epoch: 0004, loss: 3.5616 - val_loss: 4.1940; rmse: 1.8905 - rmse_val: 2.0527;  r2: 0.2471 - r2_val: 0.1867 

In [44]:
pd.DataFrame(Regression_res)['test_r2'].mean()

0.6708837737287009

In [38]:
pd.DataFrame(Regression_res)['test_r2'].mean()

0.6892917379198978

In [31]:
pd.DataFrame(Regression_res)['test_r2'].mean()

0.6687799024170524